In [13]:
import urllib.request
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
width = 13
vstat = "http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/10_2019/10010008.19B"
page = urllib.request.urlopen(vstat)
content = page.read()

In [14]:
def qwe():
    width = 13
    vstat = "http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/10_2019/10010008.19B"
    page = urllib.request.urlopen(vstat)
    content = page.read()
    data_dopolnitelniy = np.frombuffer(content[:17], dtype=np.dtype('i1'))
    data_dopolnitelniy = np.frombuffer(data_dopolnitelniy[:16], dtype=np.dtype('i2'))
    date_temp = data_dopolnitelniy[:-1]
    date_time = datetime(*date_temp).strftime("%d.%m.%Y %H:%M:%S.%f")
    numpy_data_polojitelnie = np.frombuffer(content, dtype=np.dtype('B'))[17:-14]
    numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie) / width), width)
    numpy_data_polojitelnie  = np.delete(numpy_data_polojitelnie,12,1)
    numpy_data_polojitelnie = np.frombuffer(numpy_data_polojitelnie, dtype=np.dtype('i2'))
    numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie)/6),6)
    row = numpy_data_polojitelnie.shape[0]
    column = numpy_data_polojitelnie.shape[1] + 2
    zero = np.zeros((row,1))
    massiv = np.append(zero,numpy_data_polojitelnie,axis=1) 
    numpy_data_polojitelnie = np.append(massiv,zero,axis=1)

    data_frame1 = pd.DataFrame(numpy_data_polojitelnie)

    data_frame1.head()
    for i in range(len(numpy_data_polojitelnie)):
    #     numpy_data_polojitelnie[i,0] = numpy_data_polojitelnie[i,0] // 100
        numpy_data_polojitelnie[i,1] = numpy_data_polojitelnie[i,1] / 100
        numpy_data_polojitelnie[i,2] = numpy_data_polojitelnie[i,2] / 100
        numpy_data_polojitelnie[i,3] = numpy_data_polojitelnie[i,3] / 100
        numpy_data_polojitelnie[i,4] = numpy_data_polojitelnie[i,4] / 100
        numpy_data_polojitelnie[i,5] = numpy_data_polojitelnie[i,5] / 10
        numpy_data_polojitelnie[i,6] = numpy_data_polojitelnie[i,6] / 100

    dft = pd.DataFrame((numpy_data_polojitelnie), columns=[
        'A','A','A','A','A','A','A','A'],index=pd.date_range(
        date_time, periods=len(numpy_data_polojitelnie), freq='12.5ms'))
    dft.rename_axis("limbs", axis="columns")
    dft.columns.name = 'date and time'
    dft.head()
    return dft
    

In [15]:
# %time qwe()

In [16]:
data_dopolnitelniy = np.frombuffer(content[:17], dtype=np.dtype('i1'))
data_dopolnitelniy = np.frombuffer(data_dopolnitelniy[:16], dtype=np.dtype('i2'))
date_temp = data_dopolnitelniy[:-1]
date_time = datetime(*date_temp).strftime("%d.%m.%Y %H:%M:%S.%f")
numpy_data_polojitelnie = np.frombuffer(content, dtype=np.dtype('B'))[17:-14]
numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie) / width), width)
numpy_data_polojitelnie  = np.delete(numpy_data_polojitelnie,12,1)
numpy_data_polojitelnie = np.frombuffer(numpy_data_polojitelnie, dtype=np.dtype('i2'))
numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie)/6),6)
row = numpy_data_polojitelnie.shape[0]
column = numpy_data_polojitelnie.shape[1] + 2
zero = np.zeros((row,1))
massiv = np.append(zero,numpy_data_polojitelnie,axis=1) 
numpy_data_polojitelnie = np.append(massiv,zero,axis=1)

data_frame1 = pd.DataFrame(numpy_data_polojitelnie)

data_frame1.head()

,0,1,2,3,4,5,6,7
0,0.0,233.0,-184.0,-126.0,115.0,7437.0,9343.0,0.0
1,0.0,242.0,-188.0,-122.0,102.0,7438.0,9344.0,0.0
2,0.0,232.0,-178.0,-103.0,81.0,7437.0,9343.0,0.0
3,0.0,237.0,-172.0,-105.0,71.0,7437.0,9344.0,0.0
4,0.0,237.0,-172.0,-101.0,58.0,7437.0,9344.0,0.0


In [17]:
for i in range(len(numpy_data_polojitelnie)):
#     numpy_data_polojitelnie[i,0] = numpy_data_polojitelnie[i,0] // 100
    numpy_data_polojitelnie[i,1] = numpy_data_polojitelnie[i,1] / 100
    numpy_data_polojitelnie[i,2] = numpy_data_polojitelnie[i,2] / 100
    numpy_data_polojitelnie[i,3] = numpy_data_polojitelnie[i,3] / 100
    numpy_data_polojitelnie[i,4] = numpy_data_polojitelnie[i,4] / 100
    numpy_data_polojitelnie[i,5] = numpy_data_polojitelnie[i,5] / 10
    numpy_data_polojitelnie[i,6] = numpy_data_polojitelnie[i,6] / 100

dft = pd.DataFrame((numpy_data_polojitelnie), columns=[
    'A','A','A','A','A','A','A','A'],index=pd.date_range(
    date_time, periods=len(numpy_data_polojitelnie), freq='12.5ms'))
dft.rename_axis("limbs", axis="columns")
dft.columns.name = 'date and time'
dft.head()


date and time,A,A,A,A,A,A,A,A
2019-01-10 00:08:14.000343,0.0,2.33,-1.84,-1.26,1.15,743.7,93.43,0.0
2019-01-10 00:08:14.012843,0.0,2.42,-1.88,-1.22,1.02,743.8,93.44,0.0
2019-01-10 00:08:14.025343,0.0,2.32,-1.78,-1.03,0.81,743.7,93.43,0.0
2019-01-10 00:08:14.037843,0.0,2.37,-1.72,-1.05,0.71,743.7,93.44,0.0
2019-01-10 00:08:14.050343,0.0,2.37,-1.72,-1.01,0.58,743.7,93.44,0.0


In [18]:
date_time

'01.10.2019 00:08:14.000343'

In [23]:
import pandas as pd
datelist = pd.date_range(date_time, periods=len(numpy_data_polojitelnie), freq='12.5ms').to_pydatetime()
np.insert(a, [1], [[1],[2],[3]], axis=1)

F:\Users\Apple\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: in the future the special handling of scalars will be removed from insert and raise an error
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: assignment to 0-d array

In [7]:
import pandas as pd
distance_array = []
for x in range(1,2):
    a=str(x) + ".csv"
    df = pd.read_csv(a, error_bad_lines=False, nrows=2)
    for index, row in df.iterrows():
        distance_array.append(x)
        print(index,row)
# print(sum(distance_array)/len(distance_array))

0 12;12;12;12;12;12;12    12;12;12;12;12;12;12
Name: 0, dtype: object
1 12;12;12;12;12;12;12    12;12;12;12;12;12;12
Name: 1, dtype: object
